In [2]:
import logging
import sqlite3
import json
from datetime import datetime

# import numpy as np
import pandas as pd
import numpy as np

# trading_ig
from trading_ig.rest import IGService
from trading_ig.config import config

import yfinance as yf
from datetime import timedelta
from datetime import datetime
from autoIG.utils import format_date

logging.basicConfig(
    format="%(asctime)s %(levelname)-8s %(module)-20s %(message)s",
    level=logging.INFO,
    datefmt="%Y-%m-%d %H:%M:%S",
)


In [6]:
# config
from autoIG.epics import GOLD_EPIC, SANDP_EPIC
DAYS_HISTORY = 10
STARTDATE = format_date(datetime.now() - timedelta(days=DAYS_HISTORY))
ENDDATE = format_date(datetime.now() + timedelta(days=1))
RESOLUTION = "H"


In [ ]:
def create_open_position_config(epic,size = None,direction = 'BUY'):

    market = ig_service.fetch_market_by_epic(epic)
    expiry = market.instrument['expiry']
    minsize= market.dealingRules['minDealSize']['value']
    if size is None:
        size = minsize
    else:
        if size<minsize:
            raise ValueError("Size of trade too small")
    res = {
        'currency_code':'GBP',
        'direction':direction,
        'epic':epic,
        'order_type':'MARKET',
        'expiry':expiry,
        'force_open':'false',
        'guaranteed_stop':'false',
        'size':size,
        'level':None,
        'limit_distance':None,
        'limit_level':None,
        'quote_id':None,
        'stop_level':None,
        'stop_distance':None,
        'trailing_stop':None,
        'trailing_stop_increment':None
        }
    return res

In [3]:
ig_service = IGService(config.username, config.password, config.api_key)
ig = ig_service.create_session()

2022-11-20 17:05:32 INFO     rest                 Creating new v2 session for user 'citrez' at 'https://demo-api.ig.com/gateway/deal'
2022-11-20 17:05:33 INFO     rest                 POST '/session', resp 200


In [7]:
market = ig_service.fetch_market_by_epic(GOLD_EPIC)
print(f"Keys: {list(market.keys())}")
print(json.dumps(market.instrument,indent=4))

2022-11-20 17:06:41 INFO     rest                 GET '/markets/MT.D.GC.Month2.IP', resp 200


Keys: ['instrument', 'dealingRules', 'snapshot']
{
    "epic": "MT.D.GC.Month2.IP",
    "expiry": "APR-23",
    "name": "Gold",
    "forceOpenAllowed": true,
    "stopsLimitsAllowed": true,
    "lotSize": 1.0,
    "unit": "AMOUNT",
    "type": "COMMODITIES",
    "controlledRiskAllowed": true,
    "streamingPricesAvailable": true,
    "marketId": "GC",
    "currencies": [
        {
            "code": "USD",
            "symbol": "$",
            "baseExchangeRate": 1.189065,
            "exchangeRate": 0.66,
            "isDefault": false
        },
        {
            "code": "AUD",
            "symbol": "A$",
            "baseExchangeRate": 1.782295,
            "exchangeRate": 0.45,
            "isDefault": false
        },
        {
            "code": "EUR",
            "symbol": "E",
            "baseExchangeRate": 1.15201,
            "exchangeRate": 0.9,
            "isDefault": false
        },
        {
            "code": "GBP",
            "symbol": "\u00a3",
            

In [ ]:
print(json.dumps(market.dealingRules,indent=4))

In [ ]:
print(json.dumps(market.snapshot,indent=4))

In [8]:
# see from pandas.tseries.frequencies import to_offset
# resolution = '1Min'
result = ig_service.fetch_historical_prices_by_epic(
    epic=GOLD_EPIC, start_date=STARTDATE, end_date=ENDDATE, resolution=RESOLUTION
)
print(f"Keys: {list(result.keys())}")
print(result["instrumentType"])
prices_raw = result["prices"]
print(prices_raw.shape)
prices_raw.head(3)


2022-11-20 17:07:26 INFO     rest                 GET '/prices/MT.D.GC.Month2.IP', resp 500


IGException: Server problem: status code: 500, reason: Internal Server Error

In [ ]:
# Depreciated - Use janitor clean_names()
# def standardise_column_names(df):
#     df_new = df.copy()
#     df_new.columns = ["_".join(i).lower() for i in df_new.columns]
#     return df_new

import janitor

prices = prices_raw.clean_names()
prices  = prices.reset_index()
prices.head(3)

In [ ]:
def to_hours(td):
    "Gives the number of hours diffreence between two timedeltas"
    return td.days*24+ td.seconds//3600
to_hours(prices.DateTime[136] - prices.DateTime[0])

In [ ]:
prices = prices.assign(
    days_since=lambda df: list(map(lambda x: to_hours(x), (df["DateTime"] - df["DateTime"][0])))
)
prices

In [ ]:
test_size = 0
prices = prices.assign(
    train_test=["train"] * (len(prices) - test_size) + ["test"] * test_size
)
prices.train_test.value_counts()

In [ ]:
from sklearn.linear_model import LinearRegression
train = prices.query("train_test=='train'")
test = prices.query("train_test=='test'")
X_train = train.days_since.to_numpy().reshape(-1,1)
X_test = test.days_since.to_numpy().reshape(-1,1)
y_train = train.bid_open
y_test = test.bid_open


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

linearModpipeline = Pipeline(
    [("poly", PolynomialFeatures(degree=3))]
        )

linearModpipeline.fit(X_train)
X_train_preprocessed = linearModpipeline.transform(X_train)


In [ ]:
linearMod = LinearRegression(fit_intercept=True)
linearMod.fit(X_train_preprocessed,y_train)

In [ ]:
(X_train[-1]+1).reshape(-1,1)


In [ ]:
y_predictions = linearMod.predict(linearModpipeline.transform((X_train[-1]+1).reshape(-1,1)))
y_predictions

In [ ]:
from sklearn.metrics import mean_squared_error


In [ ]:
# open_positions = ig_service.fetch_open_positions()
# open_positions = (
#     open_positions.assign(
#         direction_signed=lambda df: np.where(df.direction == "SELL", -1, 1),
#         size_signed=lambda df: df['size'] * df.direction_signed
#     )
# )
# open_positions_totals = open_positions.groupby('epic',as_index=False)['size_signed'].sum()
# open_positions_totals

def get_open_position_totals():
    open_positions = ig_service.fetch_open_positions()
    
    open_positions = (
    open_positions.assign(
        direction_signed=lambda df: np.where(df.direction == "SELL", -1, 1),
        size_signed=lambda df: df['size'] * df.direction_signed
    )
)
    open_positions_totals = open_positions.groupby('epic',as_index=False)['size_signed'].sum()
    return open_positions_totals   
get_open_position_totals()


In [ ]:
downscaling= 50
total_size = open_positions_totals[open_positions_totals.epic == GOLD_EPIC]['size_signed']
total_size = float(total_size)
size_wanted =  ( y_predictions[0]-y_train.iloc[-1] )/downscaling
print(size_wanted)
change_needed = (size_wanted-total_size).round(2)
print(f"Size wanted: {size_wanted.round(2)}")
print(f"Toal size: {total_size}")
print(f"change needed: {change_needed.round(2)}")


    

In [ ]:
if change_needed> 0 :
    print("We're buying!")
    print(position_info(GOLD_EPIC,direction = 'BUY'))
    ig_service.create_open_position(**position_info(GOLD_EPIC,size= abs(change_needed),direction = 'BUY'))
if change_needed < 0:
    print("We're selling!")
    print(position_info(GOLD_EPIC,direction = 'SELL'))
    res = ig_service.create_open_position(**position_info(GOLD_EPIC,direction = 'SELL',size = abs(change_needed)))
    print(res)
if change_needed==0:
    print("No trade needed")


In [ ]:
(
    ggplot(prices)+
    geom_point(aes('days_since','bid_open',color = 'train_test'))+
    theme(figure_size = [10,3])+
    pn.geom_point(data = test,mapping = aes('days_since',y_predictions))
)

In [ ]:
ig_service.create_open_position(**position_info(sandp_epic,direction='SELL'))

In [ ]:
open_positions = ig_service.fetch_open_positions()
open_positions

In [ ]:
open_positions.columns

In [ ]:
open_positions = (
    open_positions.assign(
        direction_signed=lambda df: np.where(df.direction == "SELL", -1, 1),
        size_signed=lambda df: df['size'] * df.direction_signed
    )
)
open_positions_totals = open_positions.groupby('epic',as_index=False)['size_signed'].sum()
open_positions_totals


In [ ]:
result = ig_service.fetch_historical_prices_by_epic(
    epic='CS.D.USCGC.TODAY.IP',
    start_date="2022-02-01",
    end_date="2022-02-03",resolution="4H"
    )
result['prices']

In [ ]:
ig

In [ ]:
msft = yf.Ticker("MSFT")

In [ ]:
# get stock info
print(msft.info.keys())
msft.info['sector']

In [ ]:
from datetime import datetime,timedelta
today =datetime.now()#.strftime("%Y-%m-%d")
three_months_ago = today+timedelta(days=-90)

def format_date(date):
    return date.strftime("%Y-%m-%d")

In [ ]:
history_raw = msft.history(
    period="1d",
    end = format_date(today),
    start = format_date(three_months_ago)
)
history_raw.head()

In [ ]:
(
    ggplot(history_raw)+
    geom_point(aes(x,y))+
    theme()
)   

In [ ]:
hist = hist_raw.reset_index().reset_index()

In [ ]:
## Fit a simple linear regression with a quadratic term to the last 3 months, and buy/ sell accordingly

In [ ]:
# import statsmodels as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:

X_poly[0:3]

In [ ]:
import plotnine as pn
from plotnine import ggplot, geom_line, aes

In [ ]:
(
    ggplot(hist.reset_index().reset_index())+
    geom_line(pn.aes('index','Close'))
    )


In [ ]:
# np.array(hist['index']).reshape(-1,1)

In [ ]:
linreg = LinearRegression()
linreg.fit(np.array(hist['index']).reshape(-1,1),  hist.Close)

In [ ]:
polyreg =  LinearRegression()
poly =  PolynomialFeatures(degree=3)
X_poly = poly.fit_transform(np.array(hist['index']).reshape(-1,1))
polyreg.fit(X_poly,hist.Close)

In [ ]:
# inputs = np.linspace(0,60,100)
# poly = PolynomialFeatures(degree = 2)
# inputs = poly.fit_transform(inputs.reshape(-1,1))

# poly = PolynomialFeatures(degree = 3)
# inputs = poly.fit_transform(inputs.reshape(-1,1))



In [ ]:
X_poly[:,1]

In [ ]:
def plot_model(model = linreg):
    inputs = np.linspace(0,60,100)
    inputs = poly.fit_transform(inputs.reshape(-1,1))

    preds= polyreg.predict(inputs)

    gg = (
        ggplot()+
        geom_line(aes(inputs[:,1],preds))+ # plot the model
        geom_line(aes(hist['index'],hist['Close']))
    )
    return gg

plot_model(polyreg)




In [ ]:
linreg.predict(np.array([[1]]))

## misc

In [ ]:
## open a possition
resp = ig_service.create_open_position(
        currency_code='GBP',
        direction='BUY',
        epic=gold_epic,
        order_type='MARKET',
        expiry="AUG-22",
        force_open='false',
        guaranteed_stop='false',
        size=0.5,
        level=None,
        limit_distance=None,
        limit_level=None,
        quote_id=None,
        stop_level=None,
        stop_distance=None,
        trailing_stop=None,
        trailing_stop_increment=None
        )
resp